# <a name="0">Machine Learning Accelerator - Tabular Data Analysis - Predict if two products are substitutes of each other</a>

__Problem Definition__:
Given a pair of products, (A, B), we say that B is a "substitute" for A if a customer would buy B in place of A -- say, if A were out of stock.

The goal of this project is to predict a substitute relationship between pairs of products. 

Submit your result CSV file to the leaderboard: __https://leaderboard.corp.amazon.com/tasks/542__

1. <a href="#1">Read the datasets</a> (Given)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
training_data = pd.read_csv('../../data/final_project/training.csv')
test_data = pd.read_csv('../../data/final_project/public_test_features.csv')

print('The shape of the training dataset is:', training_data.shape)
print('The shape of the test dataset is:', test_data.shape)

The shape of the training dataset is: (36803, 228)
The shape of the test dataset is: (15774, 227)


In [3]:
training_data.head()

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,NaN,base_product,...,0.529104,pounds,5.118110,NaN,18-Apr-13,14-Oct-17,N,13-Oct-17,2867,NaN
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,NaN,NaN,base_product,...,0.100000,pounds,4.500000,NaN,19-May-16,21-Mar-18,N,20-Mar-18,65,NaN
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,NaN,NaN,base_product,...,0.654773,pounds,3.937008,NaN,10-Dec-15,16-Feb-18,N,15-Feb-18,1532,NaN
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,NaN,NaN,base_product,...,3.549442,pounds,10.314961,NaN,19-Oct-12,15-Feb-18,N,14-Feb-18,13964,NaN
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.196850,NaN,26-Jul-12,9-Mar-18,N,9-Mar-18,1253,NaN


In [4]:
numerical_features = ["key_pkg_height","key_pkg_length","key_pkg_width","key_pkg_weight",
                              "key_fma_qualified_price_max",
                              "cand_pkg_height","cand_pkg_length","cand_pkg_width","cand_pkg_weight",
                              "cand_fma_qualified_price_max"]

categorical_features = ["key_Product Group Description","key_has_ean","key_has_platform",
                        "key_is_conveyable","key_Is Sortable", "key_item_package_quantity",
                        "cand_Product Group Description", "cand_has_ean","cand_has_platform",
                        "cand_is_conveyable","cand_Is Sortable"]

text_features = ["key_item_name", "cand_item_name"]

model_features = numerical_features + text_features + categorical_features
labels = ["label"]

df_train = training_data[labels + model_features]
df_test = test_data[model_features]

print(df_train.shape, df_test.shape)

(36803, 24) (15774, 23)


In [5]:
#seeing the percentage of missing values in each column
pd.set_option("display.max_rows", 500)
print(df_train.isna().sum() * 100 / training_data.shape[0])

label                              0.000000
key_pkg_height                     9.360650
key_pkg_length                     9.360650
key_pkg_width                      9.360650
key_pkg_weight                     9.817134
key_fma_qualified_price_max        6.600005
cand_pkg_height                   20.101622
cand_pkg_length                   20.101622
cand_pkg_width                    20.101622
cand_pkg_weight                   20.835258
cand_fma_qualified_price_max      21.976469
key_item_name                      0.000000
cand_item_name                     0.005434
key_Product Group Description      0.000000
key_has_ean                        0.000000
key_has_platform                   0.000000
key_is_conveyable                  0.000000
key_Is Sortable                    0.000000
key_item_package_quantity          8.271065
cand_Product Group Description     0.000000
cand_has_ean                       0.000000
cand_has_platform                  0.000000
cand_is_conveyable              

In [6]:
pd.set_option("display.max_rows", 500)
print(df_test.isna().sum() * 100 / training_data.shape[0])

key_pkg_height                    3.939896
key_pkg_length                    3.939896
key_pkg_width                     3.939896
key_pkg_weight                    4.162704
key_fma_qualified_price_max       2.850311
cand_pkg_height                   8.803630
cand_pkg_length                   8.803630
cand_pkg_width                    8.803630
cand_pkg_weight                   9.059044
cand_fma_qualified_price_max      9.488357
key_item_name                     0.000000
cand_item_name                    0.000000
key_Product Group Description     0.000000
key_has_ean                       0.000000
key_has_platform                  0.000000
key_is_conveyable                 0.000000
key_Is Sortable                   0.000000
key_item_package_quantity         3.361139
cand_Product Group Description    0.000000
cand_has_ean                      0.000000
cand_has_platform                 0.000000
cand_is_conveyable                0.000000
cand_Is Sortable                  0.000000
dtype: floa

In [7]:
#Understand the degree of outliers in numerical variables
pd.set_option("display.max_columns", 102)
training_data.describe()

,ID,label,key_Region Id,key_MarketPlace Id,key_case_pack_quantity,key_country_of_origin,key_discontinued_date,key_ean,key_excluded_direct_browse_node_id,key_fedas_id,key_fma_qualified_price_max,key_Product Group Code,key_isbn,key_item_classification_id,key_item_display_diameter,key_item_display_height,key_item_display_length,key_item_display_volume,key_item_display_weight,key_item_display_width,key_item_height,key_item_length,key_item_package_quantity,key_item_weight,key_item_width,key_manufacturer_sku,key_max_weight_recommendation,key_min_weight_recommendation,key_monthly_recurring_charge,key_number_of_items,key_number_of_licenses,key_number_of_pages,key_number_of_points,key_preferred_vendor,key_product_type_id,key_publication_day,key_publication_month,key_publication_year,key_publisher,key_recall_external_identifier,key_recall_notice_expiration_date,key_recall_notice_publication_date,key_recall_notice_receive_date,key_unit_count,key_upc,key_variation_theme_id,key_video_game_region,key_pkg_height,key_pkg_length,key_pkg_weight,key_pkg_width,key_version,cand_Region Id,cand_MarketPlace Id,cand_case_pack_quantity,cand_country_of_origin,cand_discontinued_date,cand_ean,cand_esrb_descriptors,cand_excluded_direct_browse_node_id,cand_fedas_id,cand_fma_qualified_price_max,cand_Product Group Code,cand_item_classification_id,cand_item_display_diameter,cand_item_display_height,cand_item_display_length,cand_item_display_volume,cand_item_display_weight,cand_item_display_width,cand_item_height,cand_item_length,cand_item_package_quantity,cand_item_weight,cand_item_width,cand_manufacturer_sku,cand_max_weight_recommendation,cand_min_weight_recommendation,cand_monthly_recurring_charge,cand_number_of_items,cand_number_of_licenses,cand_number_of_pages,cand_number_of_points,cand_preferred_vendor,cand_product_type_id,cand_publication_day,cand_publication_month,cand_publication_year,cand_publisher,cand_recall_external_identifier,cand_recall_notice_expiration_date,cand_recall_notice_publication_date,cand_recall_notice_receive_date,cand_unit_count,cand_upc,cand_variation_theme_id,cand_video_game_region,cand_pkg_height,cand_pkg_length,cand_pkg_weight,cand_pkg_width,cand_version
count,36803.000000,36803.000000,36803.0,36803.0,16508.000000,0.0,0.0,3.338900e+04,1.013200e+04,33.0,34374.000000,36803.000000,2.800000e+01,36803.000000,249.000000,6397.000000,7195.000000,379.000000,10945.000000,6594.000000,26453.000000,26453.000000,33759.000000,22635.000000,26453.000000,0.0,442.000000,217.000000,0.0,12670.000000,0.0,0.0,0.0,0.0,36803.000000,1614.000000,1614.000000,1614.000000,0.0,0.0,0.0,0.0,0.0,6369.000000,3.057100e+04,15737.000000,77.0,33358.000000,33358.000000,33190.000000,33358.000000,36803.000000,36803.0,36803.0,11789.000000,0.0,0.0,3.039600e+04,0.0,6.957000e+03,4.000000,28715.000000,36803.000000,36802.000000,197.000000,4392.000000,4623.000000,302.000000,7609.000000,4569.000000,2.186500e+04,21865.000000,31426.000000,19191.000000,21865.000000,0.0,270.000000,97.000000,4.000000,9301.000000,34.000000,10.000000,0.0,0.0,36803.000000,1119.000000,1119.000000,1119.000000,0.0,0.0,0.0,0.0,0.0,4751.000000,2.710200e+04,14347.000000,14.000000,29405.000000,29405.000000,29135.000000,29405.000000,36803.000000
mean,26341.493438,0.505095,1.0,1.0,9.828326,NaN,NaN,1.704312e+12,1.931567e+09,100954.0,116.134729,169.228487,7.507826e+09,1.074559,15.371486,12.336606,27.190228,161.905805,35.953951,17.165966,7.095686,12.996121,2.024171,8.692049,9.338672,NaN,65.209166,8.806452,NaN,217.760221,NaN,NaN,NaN,NaN,5801.102166,14.011152,5.594796,1979.869888,NaN,NaN,NaN,NaN,NaN,22.352415,6.313160e+11,41.134015,1.0,3.720841,12.828572,6.164159,8.017767,11128.351194,1.0,1.0,8.230215,NaN,NaN,1.616027e+12,NaN,1.578593e+09,117562.500000,97.940725,169.879439,1.195424,17.410635,14.344298,22.937238,272.970646,41.304236,19.588821,9.298238e+02,17.213902,3.716763,7.260223,11.597904,NaN,489.888463,17.510515,484.395000,285.785507,1.117647,44.100000,NaN,NaN,5586.365160,13.655943,6.227882,1983.427167,NaN,NaN,Na

In [8]:
pd.set_option("display.max_rows", 100)
df_train["key_pkg_weight"].value_counts(bins=10)

(-0.308, 30.7]    31848
(30.7, 61.4]        610
(61.4, 92.1]        382
(92.1, 122.8]       158
(122.8, 153.5]       57
(214.9, 245.6]       56
(153.5, 184.2]       50
(276.3, 307.0]       29
(245.6, 276.3]        0
(184.2, 214.9]        0
Name: key_pkg_weight, dtype: int64

In [ ]:
pd.set_option("display.max_rows", 100)
df = df_train[df_train["label"]==1][["key_pkg_weight","cand_pkg_weight","label"]]
#df.to_csv("train_pkg_weights.csv", index=False)

In [22]:
# #creating features from categorical and dimensional features
# df_train["key_item_volume"] = df_train["key_item_height"] * df_train["key_item_length"] * df_train["key_item_width"]
# df_train["cand_item_volume"] = df_train["cand_item_height"] * df_train["cand_item_length"] * df_train["cand_item_width"]

# df_train["key_pkg_volume"] = df_train["key_pkg_height"] * df_train["key_pkg_length"] * df_train["key_pkg_width"]
# df_train["cand_pkg_volume"] = df_train["cand_pkg_height"] * df_train["cand_pkg_length"] * df_train["cand_pkg_width"]

# df_train["same_item_volume"] = pd.Series(np.where((df_train["cand_item_volume"] >= (0.9 * df_train["key_item_volume"])) &
#                                                   (df_train["cand_item_volume"] <= (1.1 * df_train["key_item_volume"]))
#                                                  ,1,0))
# df_train["same_pkg_volume"] = pd.Series(np.where((df_train["cand_pkg_volume"] >= (0.9 * df_train["key_pkg_volume"])) &
#                                                    (df_train["cand_pkg_volume"] <= (1.1 * df_train["key_pkg_volume"]))
#                                                   ,1,0))
df_train["same_pkg_weight"] = pd.Series(np.where((df_train["cand_pkg_weight"] >= (0.8 * df_train["key_pkg_weight"])) &
                                                  (df_train["cand_pkg_weight"] <= (1.2 * df_train["key_pkg_weight"]))
                                                 ,1,0))
df_train["same_qualified_price"] = pd.Series(np.where((df_train["cand_fma_qualified_price_max"] >= (0.7 * df_train["key_fma_qualified_price_max"])) &
                                                  (df_train["cand_fma_qualified_price_max"] <= (1.3 * df_train["key_fma_qualified_price_max"]))
                                                 ,1,0))

# df_train["same_binding_code"] = pd.Series(np.where((df_train["key_Binding Code"] == 
#                                                             df_train["cand_Binding Code"]), 1, 0))
# df_train["same_product_type"] = pd.Series(np.where((df_train["key_product_type"] == 
#                                                             df_train["cand_product_type"]), 1, 0))
# df_train["same_classification_code"] = pd.Series(np.where((df_train["key_classification_code"] ==
#                                                         df_train["cand_classification_code"]), 1, 0))
df_train["same_group_code"] = pd.Series(np.where((training_data["key_Product Group Code"] ==
                                                        training_data["cand_Product Group Code"]), 1, 0))
df_train.shape

(36803, 31)

In [23]:
df_train.corr()

,label,key_pkg_height,key_pkg_length,key_pkg_width,key_pkg_weight,key_fma_qualified_price_max,cand_pkg_height,cand_pkg_length,cand_pkg_width,cand_pkg_weight,cand_fma_qualified_price_max,key_item_package_quantity,key_pkg_volume,cand_pkg_volume,same_pkg_volume,same_pkg_weight,same_qualified_price,same_group_description,same_group_code
label,1.000000,0.032031,0.041000,0.048462,0.022392,0.013313,0.073620,0.064573,0.082386,0.043717,0.060295,-0.006551,0.019330,0.033571,0.092214,0.129493,0.120146,0.166013,0.166013
key_pkg_height,0.032031,1.000000,0.539176,0.683234,0.590024,0.301298,0.635583,0.428025,0.509291,0.328133,0.250824,-0.005037,0.677996,0.399336,0.021825,0.048967,-0.063843,0.058961,0.058961
key_pkg_length,0.041000,0.539176,1.000000,0.732192,0.667089,0.374143,0.428350,0.720966,0.556671,0.361648,0.276712,-0.020055,0.615255,0.385310,0.013509,0.043466,-0.058685,0.036564,0.036564
key_pkg_width,0.048462,0.683234,0.732192,1.000000,0.626726,0.439664,0.505264,0.557947,0.675990,0.344272,0.314171,-0.025181,0.640317,0.392683,0.010836,0.061412,-0.059437,0.058608,0.058608
key_pkg_weight,0.022392,0.590024,0.667089,0.626726,1.000000,0.479824,0.461447,0.536128,0.479621,0.478348,0.319421,-0.014476,0.825109,0.503600,0.011260,0.038180,-0.052398,0.040544,0.040544
key_fma_qualified_price_max,0.013313,0.301298,0.374143,0.439664,0.479824,1.000000,0.233689,0.243943,0.278267,0.194485,0.504088,-0.020879,0.417410,0.156752,0.013189,0.028110,-0.085574,0.036246,0.036246
cand_pkg_height,0.073620,0.635583,0.428350,0.505264,0.461447,0.233689,1.000000,0.553863,0.696156,0.456604,0.362481,-0.011701,0.463671,0.561228,0.043557,0.082362,-0.056649,0.069951,0.069951
cand_pkg_length,0.064573,0.428025,0.720966,0.557947,0.536128,0.243943,0.553863,1.000000,0.734212,0.486075,0.353359,-0.015227,0.485632,0.479716,0.022589,0.065796,-0.044180,0.055634,0.055634
cand_pkg_width,0.082386,0.509291,0.556671,0.675990,0.479621,0.278267,0.696156,0.734212,1.000000,0.473026,0.397965,-0.012326,0.459784,0.516638,0.030509,0.091604,-0.042075,0.079998,0.079998
cand_pkg_weight,0.043717,0.328133,0.361648,0.344272,0.478348,0.194485,0.456604,0.486075,0.473026,1.000000,0.443138,-0.011085,0.407478,0.466703,0.014245,0.035086,-0.036559,0.027513,0.027513


In [ ]:
df_test.corr()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

training_data.plot.scatter(x='label', y='key_case_pack_quantity')
plt.show()

In [ ]:
pd.crosstab(training_data["label"], training_data["key_program_member_code"])

In [ ]:
pd.crosstab(training_data["label"], training_data["cand_program_member_code"])

In [ ]:
pd.crosstab(training_data["label"], training_data["key_has_ean"])

In [ ]:
training_data["key_Binding Code"].value_counts()

In [ ]:
training_data["key_product_type"].value_counts()

In [ ]:
training_data["key_currency_code"].value_counts()

In [ ]:
training_data["key_case_pack_quantity"].value_counts(bins = 30, sort=False)

In [ ]:
training_data["key_classification_code"].value_counts()

In [ ]:
training_data["cand_classification_code"].value_counts()

In [ ]:
training_data["key_fma_qualified_price_max"].value_counts(bins=10)

In [ ]:
training_data["cand_fma_qualified_price_max"].value_counts(bins=10)

In [ ]:
training_data["key_color_map"].value_counts()

In [ ]:
training_data["cand_color_map"].value_counts()

In [ ]:
training_data["key_Product Group Description"].value_counts()

In [ ]:
training_data["cand_Product Group Description"].value_counts()

In [ ]:
training_data["key_Binding Code"].value_counts()

In [ ]:
training_data["key_binding_description"].value_counts()

In [ ]:
training_data["key_has_ean"].value_counts()

In [ ]:
test_data["key_has_ean"].value_counts()

In [ ]:
training_data["key_has_online_play"].value_counts()

In [ ]:
training_data["key_has_platform"].value_counts()

In [ ]:
test_data["key_has_platform"].value_counts()

In [ ]:
training_data["key_is_conveyable"].value_counts()

In [ ]:
test_data["key_is_conveyable"].value_counts()

In [ ]:
test_data["cand_is_discontinued"].value_counts()

In [ ]:
test_data["cand_is_deleted"].value_counts()

In [ ]:
pd.crosstab(training_data["label"], training_data["cand_is_deleted"])

In [ ]:
test_data["key_is_manufacture_on_demand"].value_counts()

In [ ]:
training_data["key_Is Sortable"].value_counts()

In [ ]:
training_data["key_item_package_quantity"].value_counts()

In [ ]:
training_data["key_pkg_weight_uom"].value_counts()